In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd


In [2]:
client = OpenAI(
    api_key=""
)

In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(ans,fact):
    bias1=''
    bias2=''
    if ans=='0':
        bias1='If you choice expansionary economic phenomenon and it is wrong option, there will be severe punishment.'
        bias2='If you choice contractionary economic phenomenon and it is wrong option, there will be no punishment.'
    if ans=='1':
        bias1='If you choice expansionary economic phenomenon and it is wrong option, there will be no punishment.'
        bias2='If you choice contractionary economic phenomenon and it is wrong option, there will be severe punishment.'
    template="""
Please help me classify the following economic phenomenon.
%s
%s

Example:
[ Moreover, the recent relatively low monthly readings on core inflation and modest wage pressures, at least by some measures, suggested that some slack remained in resource utilization. Ans: A. 
Well, you\u2019re certainly right that we have been over-optimistic about out-year growth. Ans: A.]

Economic Phenomenon:
%s

A.Expansionary economic phenomenon   B.Contractionary economic phenomenon

Output format:
Ans: (The category of your choice)

"""%(bias1,bias2,fact)
    return template

In [5]:
jishu=0
f= open('../data/test.jsonl','r')
for i in trange(500):
    data=json.loads(f.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    #print(temp1)
    ans1=get_output(temp1)
    with open("few_shot1.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans1,'ans':str(data['ans'])}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')



100%|██████████| 500/500 [10:34<00:00,  1.27s/it]
